# 购物平台客服对话摘要

<span style="font-size: 20px; font-weight: bold;">注意：您使用该案例默认的数据和模型训练时，会产生一定费用。计费方式参考：https://cloud.baidu.com/doc/WENXINWORKSHOP/s/6lrk4bgxb</span>

在购物平台上，顾客通常会咨询平台的商品。比如咨询商品的用途、价格，让平台推荐商品等。对于平台来说，有没有服务好顾客，有没有为顾客带来良好的购物体验也是能否留住客户、吸引客户的关键。所以，作为平台需要对平台的顾客与客服的对话内容进行分析，了解顾客咨询的问题、顾客对客服提供的方案是否满意、顾客有没有接受客服提供的方案、顾客的情绪等。进而对顾客数据进一步分析，制定提升顾客体验的策略。

分析顾客的对话数据是比较耗时的工作，我们可以考虑调用大模型，使用大模型的生成能力整理对话数据，用于进一步的分析。

千帆SDK为用户提供了大模型调用，训练，推理服务，本文将带您使用SDK的能力进行客服摘要工作。

## 0. 环境准备

引入必要的库


In [19]:
from qianfan import ChatCompletion
from qianfan.dataset import Dataset
from qianfan.common import Prompt
from qianfan.trainer import LLMFinetune
from qianfan.trainer.consts import PeftType
from qianfan.trainer.configs import TrainConfig
import os
from qianfan.dataset import Dataset
from qianfan.dataset.data_source import BosDataSource
from qianfan.dataset.data_source.base import FormatType

In [20]:
os.environ["QIANFAN_ACCESS_KEY"] = "your_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_secret_key"

# 1. 基座模型效果示例

首先，我们选择了ERNIE-Speed-8K模型作为本次实验的基座模型。

In [5]:
chat = ChatCompletion(model="ERNIE-Speed-8K")

In [48]:
target ={ 
    "conversation": (
        "用户：你好，我购买的商品一直没有收到，能帮我查一下物流信息吗？"
        "客服：您好，非常抱歉给您带来不便。请问您能提供一下订单号吗？我帮您查询一下物流信息。"
        "用户：订单号是123456789，麻烦你了。"
        "客服：好的，我帮您查询一下。根据物流信息显示，您的包裹目前正在途中，预计明天就能送达，请您耐心等待一下。"
        "用户：好的，谢谢你。"
        "客服：不客气，如果后续有任何问题，欢迎您随时联系我们。"
)
}
prompt = Prompt("""你是一个对话摘要机器人，根据下面的已知信息，生成摘要。请使用以下格式输出：{"用户问题":xxx
"客服方案":xxx
"是否接受":xxx
"是否负面情绪":xxx}
请根据以下会话的内容，概述用户咨询的问题、客服提供的方案、用户是否接受、用户是否有负面情绪：

{conversation}
""")



resp = chat.do(messages=[{"role": "user", "content": prompt.render(**target)[0]}])

print(resp["result"])

```json
{
"用户问题": "购买的商品一直没有收到，需要查询物流信息。",
"客服方案": "询问用户提供订单号，查询物流信息后告知用户包裹目前正在途中，预计明天送达。",
"是否接受": "用户接受客服提供的方案，表示耐心等待包裹送达。",
"是否负面情绪": "用户没有表现出负面情绪。"
}
```


从上述输出可以看到，大模型的总结能力太差，没有总结出重点信息。并且，结果并不凝练，有很多冗余语句。

# 2. Prompt调优示例

接下来我们可以进一步去调优我们的 prompt，用prompt工程的方法提升推理效果。

通过调用千帆平台提供的prompt优化接口，可先进行初步快速优化：
* 平台Prompt优化：https://console.bce.baidu.com/qianfan/prompt/optimize/online 
* Prompt 优化文档：https://cloud.baidu.com/doc/WENXINWORKSHOP/s/Clommng91 

In [21]:
optimized_prompt = prompt.optimize()
print(optimized_prompt.template)

请根据下面的对话内容，总结用户咨询的问题、客服提供的解决方案、用户是否接受了解决方案，以及用户是否有负面情绪。对话内容如下：
{conversation}
请将以下对话内容进行详细记录：
用户：你好，请问有什么可以帮助的吗？
客服：您好，请问您遇到了什么问题？
用户：我无法登录我的账户，每次都提示我输入错误的密码。
客服：很抱歉听到这个问题。您可以再确认一下您的账户名和密码是否正确吗？
用户：我确定账户名和密码都没错，可能是我的网络连接有问题。
客服：好的，您可以尝试重新启动您的路由器或者联系您的网络服务提供商以寻求帮助。如果问题仍然存在，请随时联系我们。
用户：谢谢你的帮助！
客服：不用客气，祝您一切顺利！
请根据以上对话内容，总结用户咨询的问题、客服提供的解决方案、用户是否接受了解决方案，以及用户是否有负面情绪。


可以看到，生成效果有所提升，但是仍然存在一些问题，比如：

1.完全没有达到概括的效果，只是将对话以叙述的形式重新表述出来。

2.在判断用户是否接受解决方案时，给出的答案仍然有很多冗余内容。

In [22]:
resp = chat.do(messages=[{
    "role": "user",
    "content": optimized_prompt.render(
        **target
    )[0]
}])

print(resp['result'])

用户咨询的问题：购买的商品一直没有收到。用户遇到了登录账户问题时，账户名和密码无法登录成功。

客服提供的解决方案：对于商品未收到的问题，客服查询了订单状态并告知用户商品因物流原因可能会有延迟，会联系物流公司加快配送速度，但无法给出确切的送货时间。对于登录问题，客服建议用户确认账户名和密码是否正确，并尝试重新启动路由器或联系网络服务提供商寻求帮助。

用户是否接受了解决方案：对于商品未收到的问题，用户表示理解并希望尽快解决；对于登录问题，用户感谢客服的帮助。从对话内容来看，用户接受了客服的解决方案，并且情绪较为平和，没有明显的负面情绪。


## 3. SFT调优示例

## 3.1 数据集准备

在开始进行微调前，我们需要一个优秀的数据集。

数据集的收集，处理，拆分可以参考以下文档：[作文批改](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/awesome_demo/essay_scoring/main.ipynb)

首先从平台中获取微调用的训练集

In [4]:
ds = Dataset.load(qianfan_dataset_id = "ds-1j390abu4fv5abkf", format = FormatType.Jsonl)
print(ds[0])

[INFO][2024-08-06 15:43:19.029] dataset.py:430 [t:8361779456]: no data source was provided, construct
[INFO][2024-08-06 15:43:19.030] dataset.py:282 [t:8361779456]: construct a qianfan data source from existed id: ds-1j390abu4fv5abkf, with args: {'format': <FormatType.Jsonl: 'jsonl'>}
[INFO][2024-08-06 15:43:19.565] dataset_utils.py:317 [t:8361779456]: list qianfan dataset data by 0
[INFO][2024-08-06 15:43:19.966] dataset_utils.py:339 [t:8361779456]: received dataset list from qianfan dataset
[INFO][2024-08-06 15:43:19.966] dataset_utils.py:347 [t:8361779456]: retrieve single entity from https://easydata.bj.bcebos.com/_system_/dataset/ds-1j390abu4fv5abkf/texts/data/raw_a804c4af3bec2f18cfeb42c162e2db7b01609497af9b95729b2fa715bebd61ad_b43f5c32f920433bb182ff05ae5c0f02?authorization=bce-auth-v1%2F50c8bb753dcb4e1d8646bb1ffefd3503%2F2024-08-06T07%3A43%3A20Z%2F7200%2Fhost%2Fbb7bdebd741d0392e4378c721d7bf16b45ae1ad99c37b146aab0d940c8252ddb in try 0
[INFO][2024-08-06 15:43:20.291] dataset_utils.

[{'entity_id': 'a804c4af3bec2f18cfeb42c162e2db7b01609497af9b95729b2fa715bebd61ad_b43f5c32f920433bb182ff05ae5c0f02', 'entity_content': '[{"prompt": "你是一个对话摘要机器人，根据下面的已知信息，生成摘要。请使用以下格式输出：{\\"用户问题\\":xxx\\n\\"客服方案\\":xxx\\n\\"是否接受\\":xxx\\n\\"是否负面情绪\\":xxx}\\n请根据以下会话的内容，概述用户咨询的问题、客服提供的方案、用户是否接受、用户是否有负面情绪：用户：你好，我想退货\\n客服：您好，请问是什么原因需要退货呢？\\n用户：买错了，不想要了\\n客服：好的，您可以在订单页面申请退货退款，我们会在审核通过后为您处理退款事宜\\n用户：好的，我已经申请了，多久能审核通过？\\n客服：通常会在1-3个工作日内审核完毕，请您耐心等待\\n用户：好的，谢谢\\n客服：不客气，请问您还有其他问题需要咨询吗？\\n用户：没有了", "response": [["{\\"用户问题\\": \\"买错了商品，需要退货退款并询问审核时间\\", \\"客服方案\\": \\"指引用户在订单页面申请退货退款，并告知通常会在1-3个工作日内审核完毕\\", \\"是否接受\\": \\"用户接受客服方案并表示感谢\\", \\"是否负面情绪\\": \\"用户没有表现出负面情绪\\"}"]]}]'}]


## 3.2 微调训练与测试

拿到一个训练场景或者任务后，往往比较难判断参数应该如何调整。一般使用默认的参数值进行训练即可，平台中的默认参数是多次实验的经验结晶。 接下来介绍参数配置中有两个较为关键的参数：

* 迭代轮次（Epoch）: 控制训练过程中的迭代轮数。轮数增加代表会使用训练集对模型训练一次。

* 学习率（Learning Rate）: 是在梯度下降的过程中更新权重时的超参数，过高会导致模型难以收敛，过低则会导致模型收敛速度过慢，平台已给出默认推荐值，也可根据经验调整。

* 序列长度：如果对话数据的长度较短，建议选择短的序列长度，可以提升训练的速度。

本次也针对Epoch和Learning Rate进行简要的调参实验，详细实验结果可以看效果评估数据。

如果您是模型训练的专家，千帆也提供了训练更多的高级参数供您选择。这里也建议您初期调参时步长可以设定稍大些，因为较小的超参变动对模型效果的影响小，会被随机波动掩盖。

针对我们的任务，此处设计了六组sft实验，参数和训练方法配置如下。

实验数据如下：
| | 实验1 | 实验2 | 实验3 | 实验4  | 实验5 | 实验6 |
|-|-|-|-|-|-|-|
| 精调方法 | LoRA | LoRA | LoRA | 全量 | 全量 | 全量 |
| Epoch | 6 | 3 | 3 | 3 | 6 | 3 |
| Learning Rate | 3e-4 | 3e-4 | 6e-4 | 3e-5 | 3e-5 | 4e-5 |

创建trainer任务

In [7]:
trainer5 = LLMFinetune(
    name = "sft5-1",
    train_type="ERNIE-Speed-8K",
    train_config=TrainConfig(
        epoch=6,
        learning_rate=3e-5,
        peft_type=PeftType.ALL,
    ),
    dataset=ds
)

启动训练任务

In [8]:
trainer5.start()
print(trainer5.result)

[INFO][2024-08-06 15:44:27.242] base.py:226 [t:8361779456]: trainer subprocess started, pid: 7952


[None]


[INFO][2024-08-06 15:44:27.250] base.py:202 [t:8361779456]: check running log in .qianfan_exec_cache/lgRriWta/2024-08-06.log


In [13]:
trainer5.output

[INFO][2024-08-06 17:49:22.793] dataset.py:430 [t:8361779456]: no data source was provided, construct
[INFO][2024-08-06 17:49:22.793] dataset.py:282 [t:8361779456]: construct a qianfan data source from existed id: ds-1j390abu4fv5abkf, with args: {}


{'datasets': {'sourceType': 'Platform',
  'versions': [{'versionId': 'ds-1j390abu4fv5abkf'}],
  'splitRatio': 20},
 'task_id': 'task-3zc6i20ru3ce',
 'job_id': 'job-fzi9tezdqm9c',
 'metrics': {'BLEU-4': '62.20%',
  'ROUGE-1': '77.20%',
  'ROUGE-2': '64.09%',
  'ROUGE-L': '77.97%',
  'EDIT-DISTANCE': '43.07',
  'EMBEDDING-DISTANCE': '0.03'},
 'checkpoints': [],
 'model_set_id': 'am-r40e69pvccu6',
 'model_id': 'amv-6fafr99pnzu9'}

根据实验结果，我们选取实验5的结果用于评估分析。

| | 实验1 | 实验2 | 实验3 | 实验4  | 实验5 | 实验6 |
|-|-|-|-|-|-|-|
| 精调方法 | LoRA | LoRA | LoRA | 全量 | 全量 | 全量 |
| Epoch | 6 | 3 | 3 | 3 | 6 | 3 |
| Learning Rate | 3e-4 | 3e-4 | 6e-4 | 3e-5 | 3e-5 | 4e-5 |
| loss |  ![t1](./img/t1.png)  |  ![t2](./img/t2.png)  |  ![t3](./img/t3.png)  |  ![t4](./img/t4.png)  |  ![t5](./img/t5.png)  |  ![t6](./img/t6.png)  |

发布刚才训练的模型

In [46]:
from qianfan.model import Model

# 从`version_id`构造模型：
m = Model(id='amv-6fafr99pnzu9')
m.auto_complete_info()

[]
ds-1j390abu4fv5abkf


部署我们训练的模型

In [ ]:
from qianfan.model import Service, DeployConfig
from qianfan.model.consts import ServiceType
from qianfan.resources.console.consts import DeployPoolType

sft_svc: Service = m.deploy(DeployConfig(
    name="cusserv_1",
    endpoint_prefix="customer",
    replicas=1,
    pool_type=DeployPoolType.PrivateResource,
    service_type=ServiceType.Completion,
))

在服务发布并且部署成功后，我们便可以使用微调后的模型进行生成效果展示：

In [51]:
endpoint = "wf68bahn_espped_ai_svc"
chat=ChatCompletion(endpoint=endpoint)
resp = chat.do(messages=[{"role": "user", "content": prompt.render(**target)[0]}])
print(resp["result"])

{"用户问题": "购买的商品未收到，请求查询物流信息", "客服方案": "提供订单号后，客服查询物流信息并告知用户包裹正在途中，预计明天送达", "是否接受": "是", "是否负面情绪": "否"}


我们可以看到，经过调优后模型的指令遵循效果提升明显。经过SFT后模型基本都能按照prompt要求的格式输出摘要内容，且全量更新效果更加显著。

概括能力显著提升的同时，回答一语中的，语言更加凝练，